In [3]:
%pip install asyncio httpx nest_asyncio

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Weather API Service for Jupyter Notebook
# First, make sure to run this at the beginning of your notebook:
# %pip install httpx nest_asyncio
# %pip install ipywidgets (optional, for interactive widgets)

import httpx
import asyncio
from typing import Any
from IPython.display import display, Markdown

# Import and apply nest_asyncio to fix the event loop issue
import nest_asyncio
nest_asyncio.apply()

# Constants
NWS_API_BASE = "https://api.weather.gov"
USER_AGENT = "weather-app/1.0"

async def make_nws_request(url: str) -> dict[str, Any] | None:
    """Make a request to the NWS API with proper error handling."""
    headers = {
        "User-Agent": USER_AGENT,
        "Accept": "application/geo+json"
    }
    async with httpx.AsyncClient() as client:
        try:
            response = await client.get(url, headers=headers, timeout=30.0)
            response.raise_for_status()
            return response.json()
        except Exception as e:
            print(f"Error making request to {url}: {e}")
            return None

def format_alert(feature: dict) -> str:
    """Format an alert feature into a readable string."""
    props = feature["properties"]
    return f"""
Event: {props.get('event', 'Unknown')}
Area: {props.get('areaDesc', 'Unknown')}
Severity: {props.get('severity', 'Unknown')}
Description: {props.get('description', 'No description available')}
Instructions: {props.get('instruction', 'No specific instructions provided')}
"""

async def get_alerts_async(state: str) -> str:
    """Get weather alerts for a US state.

    Args:
        state: Two-letter US state code (e.g. CA, NY)
    """
    url = f"{NWS_API_BASE}/alerts/active/area/{state}"
    data = await make_nws_request(url)

    if not data or "features" not in data:
        return "Unable to fetch alerts or no alerts found."

    if not data["features"]:
        return "No active alerts for this state."

    alerts = [format_alert(feature) for feature in data["features"]]
    return "\n---\n".join(alerts)

async def get_forecast_async(latitude: float, longitude: float) -> str:
    """Get weather forecast for a location.

    Args:
        latitude: Latitude of the location
        longitude: Longitude of the location
    """
    # First get the forecast grid endpoint
    points_url = f"{NWS_API_BASE}/points/{latitude},{longitude}"
    points_data = await make_nws_request(points_url)

    if not points_data:
        return "Unable to fetch forecast data for this location."

    # Get the forecast URL from the points response
    forecast_url = points_data["properties"]["forecast"]
    forecast_data = await make_nws_request(forecast_url)

    if not forecast_data:
        return "Unable to fetch detailed forecast."

    # Format the periods into a readable forecast
    periods = forecast_data["properties"]["periods"]
    forecasts = []
    for period in periods[:5]:  # Only show next 5 periods
        forecast = f"""
{period['name']}:
Temperature: {period['temperature']}°{period['temperatureUnit']}
Wind: {period['windSpeed']} {period['windDirection']}
Forecast: {period['detailedForecast']}
"""
        forecasts.append(forecast)

    return "\n---\n".join(forecasts)

# Simple wrapper functions using asyncio.run() which now works thanks to nest_asyncio
def display_alerts(state):
    """Display alerts for a given state."""
    result = asyncio.run(get_alerts_async(state))
    display(Markdown(f"## Weather Alerts for {state}\n\n{result}"))
    
def display_forecast(latitude, longitude):
    """Display forecast for given coordinates."""
    result = asyncio.run(get_forecast_async(latitude, longitude))
    display(Markdown(f"## Weather Forecast for ({latitude}, {longitude})\n\n{result}"))

In [2]:
display_alerts("TX")

## Weather Alerts for TX


Event: Red Flag Warning
Area: South Central Lowlands And Southern Rio Grande Valley; Capitan And Sacramento Mountains; El Paso; Hudspeth
Severity: Severe
Description: ...CRITICAL TO EXTREME FIRE WEATHER FOR SOUTHERN NEW MEXICO AND
FAR WEST TEXAS ON WEDNESDAY AND THURSDAY...

.An upper level trough is expected to swing through the Four
Corners on Thursday aligning the jet stream across much of New
Mexico. This will develop a lee cyclone over Eastern Colorado
while deep mixing taps into higher winds aloft. Windy to very
windy conditions will combine with single-digit lowland minimum
RH values with only the highest elevations seeing minimum RH
values in the teens. When combined with very dry fuels, critical
to possibly extreme fire weather conditions are expected for
Thursday afternoon.

* AFFECTED AREA...Fire Weather Zone 055 Texas Fire Weather Zone
055 El Paso County, Fire Weather Zone 056 Texas Fire Weather
Zone 056 Hudspeth County, Fire Weather Zone 112 South Central
Lowlands and Southern Rio Grande Valley/BLM/GLZ and Fire
Weather Zone 113 Capitan and Sacramento Mountains/Lincoln
NF/LNZ.

* TIMING...From 10 AM MDT to 9 PM MDT on Thursday.

* WINDS...Southwest 25 to 35 mph with gusts up to 55 to 65 mph.

* RELATIVE HUMIDITY...As low as 4 to 7 percent.

* EXPERIMENTAL RFTI...5 to 9 or Critical to Extreme

* IMPACTS...any fires that develop will likely spread rapidly.
Outdoor burning is not recommended.
Instructions: A Red Flag Warning means that critical fire weather conditions
are either occurring now, or will shortly. A combination of
strong winds, low relative humidity, and warm temperatures can
contribute to extreme fire behavior.

---

Event: Fire Weather Watch
Area: Beaver; Hansford; Ochiltree; Lipscomb; Hutchinson; Roberts; Deaf Smith
Severity: Severe
Description: The National Weather Service in Amarillo has issued a Fire
Weather Watch for strong winds and low relative humidity, which
is in effect from Thursday morning through Thursday evening.

* Affected Area...In Oklahoma...Beaver. In Texas...Hansford...
Ochiltree...Lipscomb...Hutchinson...Roberts and Deaf Smith.

* 20 Foot Winds...Southwest 25 to 30 mph with gusts 35 to 45 mph.

* Relative Humidity...As low as 6 percent.

* Red Flag Threat Index...5 to 7.

* Timing...Thursday from 11 AM to 9 PM.
Instructions: A Fire Weather Watch means that the potential for critical fire
weather conditions exists. Listen for later forecasts and
possible red flag warnings.

---

Event: Fire Weather Watch
Area: Cimarron; Texas; Dallam; Sherman; Hartley; Moore; Oldham
Severity: Severe
Description: * Affected Area...In Oklahoma...Cimarron and Texas. In Texas...
Dallam...Sherman...Hartley...Moore and Oldham.

* 20 Foot Winds...Southwest 25 to 35 mph with gusts up to 55 mph.

* Relative Humidity...As low as 6 percent.

* Red Flag Threat Index...Across the west-central and
northwestern Texas Panhandle, 7 to 8. Across the western and
central Oklahoma Panhandle, 8 to 9.

* Timing...Thursday from 11 AM to 9 PM.
Instructions: A Fire Weather Watch means that the potential for critical fire
weather conditions exists. Listen for later forecasts and
possible red flag warnings.

---

Event: Fire Weather Watch
Area: Sacramento Foothills and Guadalupe Mountains; Chaves County Plains; Eddy Plains; Lea; Gaines; Dawson; Borden; Andrews; Martin; Loving; Winkler; Ector; Ward; Pecos; Guadalupe and Delaware Mountains; Van Horn and Highway 54 Corridor; Eastern Culberson County; Reeves County Plains; Chinati Mountains; Marfa Plateau; Davis Mountains; Davis Mountains Foothills; Central Brewster County; Presidio Valley
Severity: Severe
Description: ...FIRE WEATHER WATCH IN EFFECT FROM THURSDAY MORNING THROUGH
THURSDAY EVENING FOR 20-FT WINDS OF 20 MPH OR MORE, RELATIVE
HUMIDITY OF WIND AND LOW RELATIVE HUMIDITY OF 15% OR LESS, AND
HIGH TO EXTREME FIRE DANGER FOR SOUTHEAST NEW MEXICO AND MOST OF
WEST TEXAS...

* WINDS...Southwest 20 to 30 mph with gusts up to 55 mph.

* RELATIVE HUMIDITY...As low as 6 percent.

* RFTI...5 or critical.

* IMPACTS...

* SEVERITY...

FUELS (ERC)...70th-89th percentile...4 (out of 5).

WEATHER...Elevated...1 (out of 5).

FIRE ENVIRONMENT...5 (out of 10).
Instructions: A Fire Weather Watch means that critical fire weather conditions
are forecast to occur. Listen for later forecasts and possible
Red Flag Warnings.

---

Event: High Wind Watch
Area: Guadalupe Mountains of Eddy County; Guadalupe Mountains Above 7000 Feet; Guadalupe and Delaware Mountains
Severity: Severe
Description: * WHAT...West winds 35 to 55 mph with gusts up to 80 mph possible.

* WHERE...Guadalupe Mountains of west Texas and Southeast New Mexico.

* WHEN...From Thursday morning through Sunday afternoon.

* IMPACTS...Travel will be difficult, especially for high profile
vehicles like campers, vans, and tractor trailers. Aviation
interests may experience localized but extreme turbulence, or
strong downward airflows if flying in the near the Guadalupe
Mountains.
Instructions: Winds will be particularly hazardous at higher elevations where the
strongest winds are likely to occur. Be especially careful driving
in these mountainous areas. People driving high-profile vehicles
should strongly consider postponing travel in these areas until the
winds subside.

Use caution if flying low near the mountains as severe turbulence is
possible.

Use caution when driving as blowing dust may reduce visibility.

---

Event: Flood Warning
Area: Beauregard, LA; Calcasieu, LA; Newton, TX; Orange, TX
Severity: Severe
Description: ...The Flood Warning continues for the following rivers in Texas...
Louisiana...

Sabine River Near Deweyville

...The Flood Warning is cancelled for the following rivers in
Louisiana...

Calcasieu River Near Glenmora

Additional information is available at www.weather.gov.

* WHAT...Minor flooding is occurring and minor flooding is forecast.

* WHERE...Sabine River near Deweyville.

* WHEN...Until early Saturday afternoon.

* IMPACTS...At 25.0 feet, Lowest roads beside the river flood around
Deweyville and subject to being closed. In addition, low-lying
roads in Southwest Beauregard Parish are flooded including Robert
Clark Road. Flooding occurs on the south side of Niblett Bluff
Park with access roads to camp houses cut off around the park.
Access roads to the river in Northeastern Orange County become
flooded.

* ADDITIONAL DETAILS...
- At 9:45 AM CDT Tuesday the stage was 24.6 feet.
- Recent Activity...The maximum river stage in the 24 hours
ending at 9:45 AM CDT Tuesday was 24.7 feet.
- Forecast...The river is expected to remain near 24.6 feet
through Thursday. It is expected to fall below flood stage
early Saturday morning.
- Flood stage is 24.0 feet.
- http://www.weather.gov/safety/flood
Instructions: None

---

Event: Flood Warning
Area: Angelina, TX; Cherokee, TX; Nacogdoches, TX
Severity: Severe
Description: ...The Flood Warning continues for the following rivers in Texas...

Angelina River Near Lufkin affecting Nacogdoches, Cherokee and
Angelina Counties.

For the Angelina River...including Alto, Lufkin...Minor flooding is
forecast.

* WHAT...Minor flooding is occurring and minor flooding is forecast.

* WHERE...Angelina River near Lufkin.

* WHEN...Until further notice.

* IMPACTS...At 162.0 feet, Minor lowland to diminish and end on the
lower Angelina River.

* ADDITIONAL DETAILS...
- At 10:00 AM CDT Tuesday the stage was 163.1 feet.
- Bankfull stage is 158.5 feet.
- Recent Activity...The maximum river stage in the 24 hours
ending at 10:00 AM CDT Tuesday was 163.2 feet.
- Forecast...The river is expected to fall to 161.5 feet Sunday
morning.
- Flood stage is 161.0 feet.
- http://www.weather.gov/safety/flood
Instructions: Do not drive cars through flooded areas.
Caution is urged when walking near riverbanks.
Turn around, don't drown when encountering flooded roads. Most flood
deaths occur in vehicles.

Motorists should not attempt to drive around barricades or drive
cars through flooded areas.

For more hydrologic information, copy and paste the following website
address into your favorite web browser URL bar:
https://water.noaa.gov/wfo/SHV

The next statement will be issued Wednesday morning at 1100 AM CDT.

---

Event: Flood Warning
Area: Smith, TX; Wood, TX
Severity: Severe
Description: ...The Flood Warning continues for the following rivers in Texas...

Sabine River Near Mineola affecting Wood and Smith Counties.

For the Sabine River...including Mineola, Hawkins, Gladewater,
Longview, Beckville, Logansport...Minor flooding is forecast.

* WHAT...Minor flooding is occurring and minor flooding is forecast.

* WHERE...Sabine River near Mineola.

* WHEN...Until Thursday evening.

* IMPACTS...At 16.0 feet, Expect flooding of low river bottoms with
secondary roadways along with picnic and recreational areas
becoming inundated as well. Ranchers should move cattle and
equipment to higher ground.

* ADDITIONAL DETAILS...
- At 10:15 AM CDT Tuesday the stage was 15.6 feet.
- Bankfull stage is 14.0 feet.
- Recent Activity...The maximum river stage in the 24 hours
ending at 10:15 AM CDT Tuesday was 16.0 feet.
- Forecast...The river is expected to fall below flood stage
early Thursday morning and continue falling to 10.5 feet
Sunday morning.
- Flood stage is 14.0 feet.
- http://www.weather.gov/safety/flood
Instructions: Do not drive cars through flooded areas.
Caution is urged when walking near riverbanks.
Turn around, don't drown when encountering flooded roads. Most flood
deaths occur in vehicles.

Motorists should not attempt to drive around barricades or drive
cars through flooded areas.

For more hydrologic information, copy and paste the following website
address into your favorite web browser URL bar:
https://water.noaa.gov/wfo/SHV

The next statement will be issued Wednesday morning at 1100 AM CDT.

---

Event: Flood Warning
Area: Anderson, TX; Cherokee, TX; Houston, TX
Severity: Severe
Description: ...The Flood Warning continues for the following rivers in Texas...

Neches River Near Neches affecting Anderson, Houston and Cherokee
Counties.

For the Neches River...including Lake Palestine, Neches, Alto,
Diboll, Rockland...Minor flooding is forecast.

* WHAT...Minor flooding is occurring and minor flooding is forecast.

* WHERE...Neches River near Neches.

* WHEN...Until late Thursday night.

* IMPACTS...At 12.0 feet, Expect minor lowland flooding of the
heavily wooded floodplain. Ranchers that may have livestock and
equipment in the river bottoms should move them to higher ground.
Expect minor flooding of the boat ramp.

* ADDITIONAL DETAILS...
- At 9:30 AM CDT Tuesday the stage was 12.6 feet.
- Bankfull stage is 12.0 feet.
- Recent Activity...The maximum river stage in the 24 hours
ending at 9:30 AM CDT Tuesday was 13.0 feet.
- Forecast...The river is expected to fall below flood stage
Thursday morning and continue falling to 9.7 feet Sunday
morning.
- Flood stage is 12.0 feet.
- http://www.weather.gov/safety/flood
Instructions: Do not drive cars through flooded areas.
Caution is urged when walking near riverbanks.
Turn around, don't drown when encountering flooded roads. Most flood
deaths occur in vehicles.

Motorists should not attempt to drive around barricades or drive
cars through flooded areas.

For more hydrologic information, copy and paste the following website
address into your favorite web browser URL bar:
https://water.noaa.gov/wfo/SHV

The next statement will be issued Wednesday morning at 1045 AM CDT.

---

Event: Flood Warning
Area: Caddo, LA; Cass, TX; Harrison, TX; Marion, TX
Severity: Severe
Description: ...The Flood Warning continues for the following rivers in Texas...
Louisiana...

Big Cypress Bayou At Caddo Lake affecting Caddo, Cass, Harrison
and Marion Counties.

For the Big Cypress Bayou...including Jefferson, Caddo Lake...Minor
flooding is forecast.

* WHAT...Minor flooding is forecast.

* WHERE...Big Cypress Bayou at Caddo Lake.

* WHEN...Until late Friday night.

* ADDITIONAL DETAILS...
- There is no current observed data.
- Bankfull stage is 169.0 feet.
- Forecast...The river is expected to fall below flood stage
early Thursday afternoon and continue falling to 171.7 feet
Sunday morning.
- Flood stage is 172.0 feet.
- Flood History...This crest compares to a previous crest of
172.0 feet on 05/14/2015.
- http://www.weather.gov/safety/flood
Instructions: Do not drive cars through flooded areas.
Caution is urged when walking near riverbanks.
Turn around, don't drown when encountering flooded roads. Most flood
deaths occur in vehicles.

Motorists should not attempt to drive around barricades or drive
cars through flooded areas.

For more hydrologic information, copy and paste the following website
address into your favorite web browser URL bar:
https://water.noaa.gov/wfo/SHV

The next statement will be issued Wednesday morning at 1030 AM CDT.

---

Event: Flood Warning
Area: Gregg, TX; Smith, TX; Upshur, TX; Wood, TX
Severity: Severe
Description: ...The Flood Warning is extended for the following rivers in Texas...

Sabine River Near Gladewater affecting Upshur, Wood, Smith and
Gregg Counties.

For the Sabine River...including Mineola, Hawkins, Gladewater,
Longview, Beckville, Logansport...Minor flooding is forecast.

* WHAT...Minor flooding is occurring and minor flooding is forecast.

* WHERE...Sabine River near Gladewater.

* WHEN...Until Friday morning.

* IMPACTS...At 29.0 feet, Expect lowland flooding to continue with
oil field operations curtailed.

* ADDITIONAL DETAILS...
- At 9:15 AM CDT Tuesday the stage was 29.2 feet.
- Bankfull stage is 25.0 feet.
- Recent Activity...The maximum river stage in the 24 hours
ending at 9:15 AM CDT Tuesday was 29.6 feet.
- Forecast...The river is expected to fall below flood stage
Thursday afternoon and continue falling to 18.9 feet Sunday
morning.
- Flood stage is 26.0 feet.
- http://www.weather.gov/safety/flood
Instructions: Do not drive cars through flooded areas.
Caution is urged when walking near riverbanks.
Turn around, don't drown when encountering flooded roads. Most flood
deaths occur in vehicles.

Motorists should not attempt to drive around barricades or drive
cars through flooded areas.

For more hydrologic information, copy and paste the following website
address into your favorite web browser URL bar:
https://water.noaa.gov/wfo/SHV

The next statement will be issued Wednesday morning at 1030 AM CDT.

---

Event: Flood Warning
Area: Delta, TX; Hopkins, TX
Severity: Severe
Description: ...The Flood Warning continues for the following rivers in Texas...

South Sulphur River Near Cooper affecting Hopkins and Delta
Counties.

* WHAT...Minor flooding is occurring and minor flooding is forecast.

* WHERE...South Sulphur River near Cooper.

* WHEN...Until late Wednesday evening.

* IMPACTS...At 16.0 feet, Minor out of bank flooding will occur. A
few rural roads will begin to flood. Minor flooding will occur to
farm and ranch land adjacent to the river.

* ADDITIONAL DETAILS...
- At 8:15 PM CDT Monday the stage was 16.5 feet.
- Bankfull stage is 16.0 feet.
- Flood stage is 16.0 feet.
- Forecast...The river is expected to fall below flood stage
late Wednesday morning and continue falling to 12.5 feet
Saturday evening.
Instructions: Do not drive cars through flooded areas.
Caution is urged when walking near riverbanks.

Additional information is available at www.water.noaa.gov/wfo/FWD.


In [3]:
# Get weather forecast for San Francisco
display_forecast(37.7749, -122.4194)

## Weather Forecast for (37.7749, -122.4194)


This Afternoon:
Temperature: 63°F
Wind: 12 mph WSW
Forecast: Partly sunny. High near 63, with temperatures falling to around 61 in the afternoon. West southwest wind around 12 mph.

---

Tonight:
Temperature: 52°F
Wind: 5 to 10 mph WSW
Forecast: Mostly cloudy, with a low around 52. West southwest wind 5 to 10 mph.

---

Wednesday:
Temperature: 62°F
Wind: 5 to 12 mph SW
Forecast: Partly sunny, with a high near 62. Southwest wind 5 to 12 mph.

---

Wednesday Night:
Temperature: 52°F
Wind: 5 to 10 mph SW
Forecast: Mostly cloudy, with a low around 52. Southwest wind 5 to 10 mph.

---

Thursday:
Temperature: 63°F
Wind: 5 to 10 mph SW
Forecast: Partly sunny, with a high near 63. Southwest wind 5 to 10 mph.
